# Проект: Генерация семантических эмбеддингов карт по масти и рангу с помощью VAE

## 📌 Пересмотренная задача
**Цель**: Создание латентных представлений карт на основе их характеристик (масть и ранг) с использованием вариационного автоэнкодера.
**Особенности**:
- Входные данные: Категориальные признаки (масть и ранг) вместо изображений
- Выход: Низкоразмерные эмбеддинги, сохраняющие семантические отношения между картами

## 1. Импорт библиотек и настройка среды

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
seed = 42

## 2. Подготовка структурированного датасета

In [13]:
suit_data = torch.tensor([0, 1, 2, 3] * 13)  # Все масти по порядку
rank_data = torch.tensor(list(range(13)) * 4)  # Все ранги по порядку
dataset = TensorDataset(suit_data, rank_data)

## 🧠 Архитектура автоэнкодера для генерации эмбеддингов

**Описание**:
Реализация стандартного автоэнкодера для преобразования пары (масть, ранг) в семантические эмбеддинги. Модель состоит из:
- **Энкодера**: Преобразует категориальные признаки в плотное векторное представление
- **Декодера**: Восстанавливает исходные характеристики из эмбеддинга
- **Бутылочного горлышка**: Низкоразмерное представление карты (эмбеддинг)

**Ключевые особенности**:
- Отдельные эмбеддинг-слои для масти и ранга
- Линейные слои с активацией ReLU
- Размерность эмбеддинга: 4 измерений
- Выходы декодера: логиты для масти (4 класса) и ранга (13 классов)

In [27]:
class CardAutoencoder(nn.Module):
    def __init__(self, embedding_dim=4):
        super(CardAutoencoder, self).__init__()
        self.embedding_dim = embedding_dim

        # Эмбеддинги для входных признаков
        self.suit_embedding = nn.Embedding(4, 4)    # 4 масти → 4D
        self.rank_embedding = nn.Embedding(13, 13)   # 13 рангов → 8D

        # Энкодер
        self.encoder = nn.Sequential(
            nn.Linear(17, 32),  # 4 (масть) + 8 (ранг) = 12 входов
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, embedding_dim)  # Бутылочное горлышко
        )

        # Декодер
        self.decoder = nn.Sequential(
            nn.Linear(embedding_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU()
        )

        # Выходные слои
        self.suit_decoder = nn.Linear(32, 4)   # Восстановление масти
        self.rank_decoder = nn.Linear(32, 13)  # Восстановление ранга

    def forward(self, suit, rank):
        # Входные эмбеддинги
        suit_emb = self.suit_embedding(suit)
        rank_emb = self.rank_embedding(rank)
        x = torch.cat((suit_emb, rank_emb), dim=1)

        # Кодирование
        embedding = self.encoder(x)

        # Декодирование
        decoded = self.decoder(embedding)

        # Восстановление характеристик
        suit_logits = self.suit_decoder(decoded)
        rank_logits = self.rank_decoder(decoded)

        return suit_logits, rank_logits, embedding


## 🧠 Определение функции потерь и оптимизатора

**Цель:**
Реализовать механизм обучения автоэнкодера, включая:

- Функцию потерь для оценки качества реконструкции масти и ранга

- Оптимизатор для обновления параметров модели

- Метрики для мониторинга процесса обучения

**Ключевые компоненты:**

- Функция потерь: Комбинация двух кросс-энтропийных потерь (для масти и ранга)

- Оптимизатор: SGD с настраиваемой скоростью обучения

- Метрики: Точность предсказания масти/ранга

In [16]:
def composite_loss(suit_logits, rank_logits, suit_true, rank_true):
    """
    Вычисляет комбинированную потерю:
    - Кросс-энтропия для масти
    - Кросс-энтропия для ранга
    """
    suit_loss = F.cross_entropy(suit_logits, suit_true)
    rank_loss = F.cross_entropy(rank_logits, rank_true)
    return suit_loss + rank_loss

def calculate_accuracy(logits, labels):
    predictions = torch.argmax(logits, dim=1)
    return (predictions == labels).float().mean()

In [50]:
embedding_dim = 2
model = CardAutoencoder(embedding_dim)
print(f"Параметры модели: {sum(p.numel() for p in model.parameters()):,}")
print(f"Размерность эмбеддинга: {embedding_dim}")
dataloader = DataLoader(dataset, batch_size=52, shuffle=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CardAutoencoder(embedding_dim=8).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1)
num_epochs = 1000

for epoch in range(num_epochs):
    total_loss = 0.0
    suit_acc, rank_acc = 0.0, 0.0

    for batch_suit, batch_rank in dataloader:
        batch_suit, batch_rank = batch_suit.to(device), batch_rank.to(device)

        # Прямой проход
        suit_logits, rank_logits, _ = model(batch_suit, batch_rank)

        # Расчет потерь
        loss = composite_loss(suit_logits, rank_logits, batch_suit, batch_rank)

        # Обратное распространение
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Сбор статистики
        total_loss += loss.item()
        suit_acc += calculate_accuracy(suit_logits, batch_suit).item()
        rank_acc += calculate_accuracy(rank_logits, batch_rank).item()

    # Средние значения за эпоху
    avg_loss = total_loss / len(dataloader)
    avg_suit_acc = suit_acc / len(dataloader) * 100
    avg_rank_acc = rank_acc / len(dataloader) * 100

    print(f"Epoch [{epoch+1}/{num_epochs}] | "
          f"Loss: {avg_loss:.4f} | "
          f"Suit Acc: {avg_suit_acc:.2f}% | "
          f"Rank Acc: {avg_rank_acc:.2f}%")

    if avg_suit_acc == 100 and avg_rank_acc == 100:
        break

print("Обучение завершено!")

Параметры модели: 2,476
Размерность эмбеддинга: 2
Epoch [1/1000] | Loss: 3.9615 | Suit Acc: 25.00% | Rank Acc: 7.69%
Epoch [2/1000] | Loss: 3.9609 | Suit Acc: 25.00% | Rank Acc: 7.69%
Epoch [3/1000] | Loss: 3.9602 | Suit Acc: 25.00% | Rank Acc: 7.69%
Epoch [4/1000] | Loss: 3.9597 | Suit Acc: 25.00% | Rank Acc: 7.69%
Epoch [5/1000] | Loss: 3.9591 | Suit Acc: 25.00% | Rank Acc: 7.69%
Epoch [6/1000] | Loss: 3.9586 | Suit Acc: 25.00% | Rank Acc: 7.69%
Epoch [7/1000] | Loss: 3.9582 | Suit Acc: 25.00% | Rank Acc: 7.69%
Epoch [8/1000] | Loss: 3.9577 | Suit Acc: 25.00% | Rank Acc: 7.69%
Epoch [9/1000] | Loss: 3.9573 | Suit Acc: 25.00% | Rank Acc: 7.69%
Epoch [10/1000] | Loss: 3.9569 | Suit Acc: 25.00% | Rank Acc: 7.69%
Epoch [11/1000] | Loss: 3.9566 | Suit Acc: 25.00% | Rank Acc: 7.69%
Epoch [12/1000] | Loss: 3.9562 | Suit Acc: 25.00% | Rank Acc: 7.69%
Epoch [13/1000] | Loss: 3.9559 | Suit Acc: 25.00% | Rank Acc: 7.69%
Epoch [14/1000] | Loss: 3.9556 | Suit Acc: 25.00% | Rank Acc: 7.69%
Epoch [